* Note that I had trouble transforming the Spark DataFrame into a format that could be used by Scikit Learn...so I switched back to pandas.

* Only run this first box if you need PySpark.

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://se

#### The next set of code reflects different methods we tried for importing in the data:

* First, we tried to read from the rds postgres database via PySpark, but could not get it working correctly.

* Next, we tried sqlalchemy, but it was slow and RAM intensive.

* Finally, we switched to putting the data on the Google Drive, which accelerated the process and reduced RAM requirements.

*Relictual code has been commmented out.*

In [1]:
# import psycopg2
import pandas as pd
import numpy as np
# from pyspark.sql import SparkSession

In [ ]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("airline_ML").getOrCreate()

In [ ]:
# df = spark.read.csv('/content/drive/MyDrive/Resources/main_airline.csv', header=True)

In [2]:
df_list = []
col_list = ["Mkt_Ccode", "Dep_Airport", "Arr_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block", "Arr_Delay_Time_Actual_Flag"]
names = ["main_airline.csv", "main_airline_2018.csv"]

for i in range(2):
   url = f"/content/drive/MyDrive/Resources/{names[i]}"
   df_list.append(pd.read_csv(url, usecols = col_list, low_memory = False))

df = pd.concat(df_list, ignore_index = True)

del df_list


#### If you subsample the data, some of the rare airports will have dummified columns with all zeroes!

* You will need a sample size of 300k plus to ensure that most of the airports are included.

* Even for relatively large sample sizes, airports IFP and YNG are often excluded.

* After subsampling, the column sums need to be checked to ensure that there are no empty columns.

In [ ]:
df.Arr_Airport.value_counts()

ORD    791055
ATL    786841
DFW    585330
DEN    552478
CLT    502557
        ...  
BFM       158
ROP       151
AKN       124
IFP        45
YNG         2
Name: Arr_Airport, Length: 376, dtype: int64

In [ ]:
df.Dep_Airport.value_counts()

ORD    791106
ATL    786785
DFW    585368
DEN    552409
CLT    502572
        ...  
BFM       158
ROP       151
AKN       124
IFP        45
YNG         2
Name: Dep_Airport, Length: 376, dtype: int64

In [3]:
df = df.loc[~df['Dep_Airport'].isin(['IFP', 'YNG'])]
df = df.loc[~df['Arr_Airport'].isin(['IFP', 'YNG'])]

In [ ]:
# Python SQL toolkit and Object Relational Mapper
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, inspect, func

In [ ]:
# SQL_USERNAME = "postgres" #your personal username goes here
# SQL_PASSWORD = "airlineproject" #your personal password goes here
# DB_NAME = "airline_db" #The default name of the DB
# DB_HOST = "airline-db.c0dqcvhs1y58.us-east-2.rds.amazonaws.com"
# PORT = 5432
# database_path = f"postgresql://{SQL_USERNAME}:{SQL_PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}"
# SQLALCHEMY_TRACK_MODIFICATIONS = False

In [ ]:
# engine = create_engine(database_path)
# connection = engine.connect()

In [ ]:
# Collect the names of tables within the database
# inspector = inspect(engine)
# inspector.get_table_names()

['main_airline', 'sample']

In [ ]:
# Using the inspector to print the column names within the 'main_airline' table and its types
# columns = inspector.get_columns('main_airline')
# for column in columns:
#     print(column["name"], column["type"])

index BIGINT
Mkt_Ccode TEXT
Mkt_Fl_No TEXT
Dep_Airport TEXT
Arr_Airport TEXT
Date BIGINT
Day_Week TEXT
Sched_Dep_Time_OAG BIGINT
Gate_Dep_Time_Actual BIGINT
Sched_Arr_Time_OAG BIGINT
Gate_Arr_Time_Actual BIGINT
Sched_Elapsed_Time_CRS BIGINT
Actual_Gate_Gate_Time BIGINT
Dep_Delay_Time_Actual BIGINT
Arr_Delay_Time_Actual BIGINT
Elapsed_Time_Diff_Actual BIGINT
Wheels_Off_Time_Actual BIGINT
Wheels_On_Time_Actual BIGINT
Tail_Number TEXT
Cancel_Code TEXT
Mins_Late_Delay_Code_E_Carrier BIGINT
Mins_Late_Delay_Code_F_Weather BIGINT
Mins_Late_Delay_Code_G_NAS BIGINT
Mins_Late_Delay_Code_H_Security BIGINT
Mins_Late_Delay_Code_I_Late_Arr_Flight BIGINT
First_Gate_Dep_Time BIGINT
Total_Ground_Away_From_Gate BIGINT
Longest_Ground_Time_Away_From_Gate BIGINT
No_Landings_At_Div_Airport BIGINT
month TEXT
Arr_Delay_Time_Actual_Flag BIGINT
Sched_Dep_Time_OAG_Block TEXT


In [ ]:
# Create empty list
# df_list = []  

# Create empty dataframe
# dfs = pd.DataFrame()  

# Start Chunking
# for chunk in pd.read_sql("SELECT * FROM main_airline", connection, chunksize = 500000):

    # Start Appending Data Chunks from SQL Result set into List
    # df_list.append(chunk)

# Start appending data from list to dataframe
# dfs = pd.concat(df_list, ignore_index=True)

# del df_list

#### If you are taking a subsample from the data...
* Subset the columns (including the target), dummify, subsample, then split out the targets and features (so they have the same length).

#### If you are using the entire data set...
* Split X and y, then dummify X.

In [4]:
# If you are taking a subsample...

X = df[["Arr_Delay_Time_Actual_Flag", "Mkt_Ccode", "Dep_Airport", "Arr_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block"]]

In [ ]:
# If you are using the entire data set...

# y1 = df[["Arr_Delay_Time_Actual_Flag"]]
# y2 = dfs[["Arr_Delay_Time_Actual"]]
# X = df[["Mkt_Ccode", "Dep_Airport", "Arr_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block"]]
# X = dfs[["Mkt_Ccode", "Dep_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block"]]
# X = dfs[["Mkt_Ccode", "Day_Week", "month", "Sched_Dep_Time_OAG_Block"]]

Dummify via pandas:

In [5]:
X_dummy = pd.get_dummies(X)

In [6]:
X_dummy.shape

(15949502, 787)

Subsample from the dummified dataframe:

In [7]:
X_dummy2 = X_dummy.sample(400000)

In [8]:
X_dummy2.shape

(400000, 787)

Check to make sure there are no columns with all zeroes:

In [9]:
dummy_count = X_dummy2.sum().to_list()

In [10]:
li_new = [i for i, element in enumerate(dummy_count) if element == 0]

print(li_new)

[]


In [11]:
y1 = X_dummy2[["Arr_Delay_Time_Actual_Flag"]]
X_dummy2.drop(columns = ["Arr_Delay_Time_Actual_Flag"], axis = 1, inplace = True)

In [32]:
y1.shape

(400000, 1)

I tried to see if I could use gc to clear memory, but it did not seem to have much of an effect.

In [ ]:
# import gc
# del df
# gc.collect()
# df = pd.DataFrame() 

In [12]:
from sklearn.metrics import *

#### Gradient Boosted Classfier

Warning: This took about six hours to run (using cross validation).

*Note that since I am evaluating perforamnce via cross validation, there is no need to train test split.*

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats = 3, random_state = 1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_dummy2, y1, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
from numpy import mean
from numpy import std

In [ ]:
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.657 (0.000)


#### Train test split:

In [154]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_dummy2, y1, random_state = 45)

#### Logit with stochastic gradient descent:

In [12]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import SGDClassifier

In [ ]:
clf = SGDClassifier(loss = "log")

In [ ]:
clf.fit(X_train, np.ravel(y_train, order = "c"))

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
pred_prob = clf.predict_proba(X_test)

In [ ]:
roc_auc = roc_auc_score(y_test, pred_prob[:,1])
print(f"AUC: {round(roc_auc, 4)}")

AUC: 0.6019


In [21]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'alpha': [10, 7, 5, 1, 0.1]}
grid = GridSearchCV(clf, param_grid, verbose=3)

Again note I am using the full test set here because of CV.

In [ ]:
grid.fit(X_dummy2, np.ravel(y1, order = "c"))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] alpha=10 ........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................ alpha=10, score=0.654, total=  16.8s
[CV] alpha=10 ........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s remaining:    0.0s


[CV] ............................ alpha=10, score=0.654, total=  16.4s
[CV] alpha=10 ........................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.3s remaining:    0.0s


[CV] ............................ alpha=10, score=0.654, total=  16.3s
[CV] alpha=10 ........................................................
[CV] ............................ alpha=10, score=0.346, total=  19.8s
[CV] alpha=10 ........................................................
[CV] ............................ alpha=10, score=0.346, total=  19.7s
[CV] alpha=7 .........................................................
[CV] ............................. alpha=7, score=0.654, total=  16.1s
[CV] alpha=7 .........................................................
[CV] ............................. alpha=7, score=0.654, total=  16.1s
[CV] alpha=7 .........................................................
[CV] ............................. alpha=7, score=0.654, total=  21.4s
[CV] alpha=7 .........................................................
[CV] ............................. alpha=7, score=0.346, total=  21.5s
[CV] alpha=7 .........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  7.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='log', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l1', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [10, 7, 5, 1, 0.1]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=3)

In [ ]:
print(grid.best_params_)

{'alpha': 5}


Logit with radial basis function and stochastic gradient descent.

In [ ]:
from sklearn.kernel_approximation import RBFSampler

In [ ]:
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)
clf = SGDClassifier(loss = "log")
clf.fit(X_features, y_train)
clf.score(X_features, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.654712

In [ ]:
X_features2 = rbf_feature.fit_transform(X_test)
pred_prob = clf.predict_proba(X_features2)

In [ ]:
roc_auc = roc_auc_score(y_test, pred_prob[:,1])
print(f"AUC: {round(roc_auc, 4)}")

AUC: 0.5025


In [ ]:
clf.score(X_features2, y_test)

0.65336

Support vector machine:

Do not run this, especially  with the grid search.

In [ ]:
from sklearn.svm import SVC 

In [ ]:
model = SVC()

In [ ]:
param_grid = {'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
grid.fit(X_dummy2, y1)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] kernel=rbf ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Neural Network (USE THIS!)

Start by encoding the target from binary to categorical.

In [13]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

In [14]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(np.ravel(y1, order = "c"))
encoded_y1 = label_encoder.transform(np.ravel(y1, order = "c"))

In [15]:
# Step 2: Convert encoded labels to one-hot-encoding
y1_categorical = to_categorical(encoded_y1)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import tensorflow as tf

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier

Tune the learning rate and momentum.

In [155]:
def create_model(learning_rate = 0.01):
    model = Sequential()
    model.add(Dense(40, input_dim = 786))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
#     model.add(Dropout(0.1))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(20))
    model.add(Activation('relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(2))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='categorical_crossentropy', optimizer = tf.optimizers.Adam(learning_rate = learning_rate), metrics=['accuracy', 'AUC']) 
    return model

In [157]:
model = KerasClassifier(build_fn = create_model, epochs = 30, batch_size = 32)

learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate = learning_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid)
grid_result = grid.fit(X_dummy2, y1_categorical)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/30
10000/10000 [==============================] - 28s 3ms/step - loss: 0.6290 - accuracy: 0.6584 - auc: 0.7024
Epoch 2/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6232 - accuracy: 0.6628 - auc: 0.7111
Epoch 3/30
10000/10000 [==============================] - 28s 3ms/step - loss: 0.6191 - accuracy: 0.6660 - auc: 0.7168
Epoch 4/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6155 - accuracy: 0.6690 - auc: 0.7217
Epoch 5/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6120 - accuracy: 0.6716 - auc: 0.7262
Epoch 6/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6093 - accuracy: 0.6738 - auc: 0.7297
Epoch 7/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6067 - accuracy: 0.6760 - auc: 0.7328
Epoch 8/30
10000/10000 [==============================] - 27s 3ms/step - loss: 0.6045 - accuracy: 0.6773 - auc: 0.7356
Epoch 9/30
10000/10000 [========================

#### Grid search epochs

In [19]:
def create_model():
    model = Sequential()
    model.add(Dense(40, input_dim = 786))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
#     model.add(Dropout(0.1))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(20))
    model.add(Activation('relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(2))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='categorical_crossentropy', optimizer = tf.optimizers.Adam(learning_rate = 0.01), metrics=['accuracy', 'AUC']) 
    return model

In [22]:
model = KerasClassifier(build_fn = create_model, batch_size = 32)

epochs = [10, 30, 50]
param_grid = dict(epochs = epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid)
grid_result = grid.fit(X_dummy2, y1_categorical)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/10
10000/10000 [==============================] - 21s 2ms/step - loss: 0.6310 - accuracy: 0.6575 - auc: 0.6997
Epoch 2/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6264 - accuracy: 0.6601 - auc: 0.7066
Epoch 3/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6235 - accuracy: 0.6622 - auc: 0.7110
Epoch 4/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6211 - accuracy: 0.6639 - auc: 0.7143
Epoch 5/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6186 - accuracy: 0.6663 - auc: 0.7176
Epoch 6/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6169 - accuracy: 0.6677 - auc: 0.7199
Epoch 7/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6151 - accuracy: 0.6693 - auc: 0.7221
Epoch 8/10
10000/10000 [==============================] - 19s 2ms/step - loss: 0.6136 - accuracy: 0.6704 - auc: 0.7242
Epoch 9/10
10000/10000 [========================

In [45]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(np.ravel(y_train, order = "c"))
label_encoder.fit(np.ravel(y_test, order = "c"))
encoded_y_train = label_encoder.transform(np.ravel(y_train, order = "c"))
encoded_y_test = label_encoder.transform(np.ravel(y_test, order = "c"))

In [49]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [129]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

In [97]:
sgd = SGD(learning_rate = 0.2, momentum = 0.2)

In [150]:
model = Sequential()
model.add(Dense(units = 40, activation = 'relu', input_dim = 786))
model.add(Dense(units = 20, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))

In [151]:
# Compile the model

opt = tf.optimizers.Adam(learning_rate = 0.001)

model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['AUC', 'accuracy'],)

In [152]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train_categorical,
    batch_size = 32,
    epochs = 30,
    shuffle = True,
    verbose = 2
)

Epoch 1/100
9375/9375 - 21s - loss: 0.6297 - auc: 0.7015 - accuracy: 0.6577
Epoch 2/100
9375/9375 - 20s - loss: 0.6234 - auc: 0.7108 - accuracy: 0.6619
Epoch 3/100
9375/9375 - 20s - loss: 0.6190 - auc: 0.7170 - accuracy: 0.6649
Epoch 4/100
9375/9375 - 21s - loss: 0.6149 - auc: 0.7224 - accuracy: 0.6682
Epoch 5/100
9375/9375 - 21s - loss: 0.6115 - auc: 0.7268 - accuracy: 0.6708
Epoch 6/100
9375/9375 - 21s - loss: 0.6086 - auc: 0.7304 - accuracy: 0.6729
Epoch 7/100
9375/9375 - 20s - loss: 0.6057 - auc: 0.7339 - accuracy: 0.6754
Epoch 8/100
9375/9375 - 20s - loss: 0.6033 - auc: 0.7368 - accuracy: 0.6771
Epoch 9/100
9375/9375 - 21s - loss: 0.6012 - auc: 0.7392 - accuracy: 0.6787
Epoch 10/100
9375/9375 - 21s - loss: 0.5993 - auc: 0.7414 - accuracy: 0.6799
Epoch 11/100
9375/9375 - 21s - loss: 0.5974 - auc: 0.7436 - accuracy: 0.6815
Epoch 12/100
9375/9375 - 21s - loss: 0.5959 - auc: 0.7452 - accuracy: 0.6825
Epoch 13/100
9375/9375 - 21s - loss: 0.5943 - auc: 0.7470 - accuracy: 0.6837
Epoch 14

In [153]:
model.evaluate(X_test, y_test_categorical)

3125/3125 [==============================] - 8s 2ms/step - loss: 0.6875 - auc: 0.6725 - accuracy: 0.6365


[0.6874668598175049, 0.6724809408187866, 0.6365100145339966]